# Import

In [3]:
from __future__ import print_function
from multiprocessing.pool import ThreadPool

import pylab as pl
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil


# 1. Select ROI

In [1]:
def crop_image_from_gray(img,tol=7):

    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]

        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img


def circle_crop_single(img):   

    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)

    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 


def circle_crop(File):   
    
    InputFile  =  '/home/john/network/Fundus/project/samples/FundusData_20230915/' + File
    img = cv2.imread(InputFile)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)

    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 

In [2]:
File_route = '/home/john/network/Fundus/project/samples/FundusData_20230915/'
ROI_File = '/home/john/network/Fundus/project/dataset/Select_ROI/'

In [ ]:
dir_list = os.listdir(File_route)

for i in dir_list:
    ImageCut = circle_crop(i)
    output_path = os.path.join(ROI_File, "ROI_{}".format(i))
    cv2.imwrite(output_path, ImageCut)

print("Done")

# 2. Image Resolution counting and classification

In [5]:
input_directory = '/home/john/network/Fundus/project/dataset/Select_ROI/'
output_directory = '/home/john/network/Fundus/project/dataset/Diff_resolution'
os.makedirs(output_directory, exist_ok=True)

resolution_dict = {}

for filename in os.listdir(input_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check image is JPG or JPEG or PNG
        input_path = os.path.join(input_directory, filename)

        # Loading image
        image = cv2.imread(input_path)

        #Get image height & width
        height, width, _ = image.shape

        #Calculate image resolution (height * width)
        resolution = (width, height)

        # Save image into different files
        resolution_str = f"{width}x{height}"
        resolution_folder = os.path.join(output_directory, resolution_str)
        os.makedirs(resolution_folder, exist_ok=True)

        # Copy images to different file
        output_path = os.path.join(resolution_folder, filename)
        cv2.imwrite(output_path, image)

        # Counting each files numbers
        if resolution_str in resolution_dict:
            resolution_dict[resolution_str] += 1
        else:
            resolution_dict[resolution_str] = 1

# Print

print("="*20)

for resolution, count in resolution_dict.items():
    print(f"Resolution  {resolution} count : {count}")


Resolution  1571x1569 count : 333
Resolution  1570x1569 count : 83
Resolution  1564x1569 count : 11
Resolution  1569x1569 count : 23
Resolution  1567x1569 count : 5
Resolution  1568x1569 count : 18
Resolution  1559x1569 count : 10
Resolution  1551x1551 count : 5
Resolution  1565x1569 count : 8
Resolution  1553x1551 count : 39
Resolution  1563x1569 count : 6
Resolution  1554x1569 count : 1
Resolution  1566x1569 count : 13
Resolution  2137x2137 count : 2
Resolution  1562x1569 count : 10
Resolution  2139x2137 count : 9
Resolution  1553x1552 count : 28
Resolution  1553x1553 count : 7
Resolution  1551x1550 count : 14
Resolution  1560x1569 count : 5
Resolution  2445x2445 count : 2
Resolution  1163x1163 count : 2
Resolution  1556x1569 count : 4
Resolution  1561x1569 count : 7
Resolution  2352x2357 count : 2
Resolution  1551x1549 count : 6
Resolution  1548x1569 count : 2
Resolution  2359x2357 count : 8
Resolution  1549x1569 count : 2
Resolution  1543x1569 count : 2
Resolution  1555x1553 count 

# 3. Resize 1571x1569 > 1570x1569

In [17]:
input_folder = '/home/john/network/Fundus/project/dataset/Used/1571x1569'
output_folder = '/home/john/network/Fundus/project/dataset/Used/All'

target_width = 1570
target_height = 1569

for filename in os.listdir(input_folder):
    if filename.endswith((".jpg")):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        image = cv2.imread(input_path)

        if image is not None:
            resized_image = cv2.resize(image, (target_width, target_height))
            cv2.imwrite(output_path, resized_image)
            print(f"Resized and saved: {output_path}")
            
        else:
            print(f"Failed to read image: {input_path}")

Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_9531160-02-R-20160302.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4288688-01-L-20190513.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4274556-01-L-20220411.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_8502518-04-L-20180209.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_7205989-01-R-20210330.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4019674-10-L-20180509.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_1049032-09-R-20210804.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4170375-03-L-20150917.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4114625-01-R-20150722.jpg
Resized and saved: /home/john/network/Fundus/project/dataset/Used/All/ROI_4539889-12-R-20230104.jpg


# 4. CLAHE

In [36]:
input_File = '/home/john/network/Fundus/project/dataset/Used/All/'
CLAHE_File = '/home/john/network/Fundus/project/dataset/Used/CLAHE/'

In [37]:
def clahe(path,file):
    InputImage = path + file

    image = cv2.imread(InputImage)

    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # 3 channel split
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    # More light 
    enhanced_l_channel = clahe.apply(l_channel)

    # Merge
    enhanced_lab_image = cv2.merge((enhanced_l_channel, a_channel, b_channel))

    # Turn back
    enhanced_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2BGR)

    return enhanced_image

In [38]:
dir_list = os.listdir(input_File)
path = input_File

for i in dir_list:
    Image_CLAHE = clahe(path,i)
    output_path = os.path.join(CLAHE_File, "CLAHE_{}".format(i))
    cv2.imwrite(output_path, Image_CLAHE)

print("Done")

Done


# 5. Select Not Blurry Image

In [83]:
# 需要多過濾幾次

def  Light(path,file):
    InputFile  =  path + file
    img = cv2. imread (InputFile)

    light = 150  # dark = 85 , light = 150

    gray_img = cv2.cvtColor (img, cv2.COLOR_BGR2GRAY)
    img_shape = gray_img. shape

    height, width = img_shape[0], img_shape [1]
    size = gray_img.size

    hist = cv2. calcHist ([gray_img], [0], None, [256], [0, 256])

    min = 0

    reduce_matrix = np. full((height, width), light) #128

    shift_value = gray_img - reduce_matrix
    shift_sum = np.sum(shift_value)

    PixelOffset = shift_sum / size

    for i in range (256):
        min += (abs( i - light - PixelOffset)  * hist[i])

    indicator = abs (min / size)
    ChangeIndicator = abs (PixelOffset) / indicator

    if ChangeIndicator[0] > 1:
        if PixelOffset > 0:
            return ("Too light")
        else:
            return ("Too dark")
    else:
        return ("Normal")

In [84]:
ImagePath = '/home/john/network/Fundus/project/dataset/Used/CLAHE/'
FirstRoundResult = '/home/john/network/Fundus/project/dataset/Used/FirstRoundResult/'  # != dark 
NotBlurryPath = '/home/john/network/Fundus/project/dataset/Used/Too light/'

In [86]:
dir_list = os.listdir(FirstRoundResult)
# FirstRoundResult , ImagePath
path = FirstRoundResult                                                             
# FirstRoundResult , ImagePath
for i in dir_list:
    ImageFilter = Light(path,i)
    
    if ImageFilter == "Normal" :

        source_path = os.path.join(FirstRoundResult, i)            
        #  ImagePath,  FirstRoundResult      
        
        destination_path = os.path.join(NotBlurryPath, i)               
        # FirstRoundResult , NotBlurryPath


        shutil.move(source_path, destination_path)

print("Done")

Done


# 6.Auto Mask Generator

In [ ]:
# Change diff File
# source_folder = '/home/john/network/Fundus/project/dataset/Used/CLAHE/'
# outputFile = '/home/john/network/Fundus/project/dataset/mask/'

In [4]:
source_folder = '/home/john/network/Fundus/project/dataset/785_data/'
outputFile = '/home/john/network/Fundus/project/dataset/785_mask/'

In [5]:
dir_list = os.listdir(source_folder)

for i in dir_list:

    srcImg = cv2.imread(source_folder +'/' +  i,cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    grountruth = cv2.imread(source_folder +'/' +  i,cv2.IMREAD_GRAYSCALE)
    grayImg = cv2.split(srcImg)[1]

    # 提取眼膜
    ret0,th0 = cv2.threshold(grayImg,5,255,cv2.THRESH_BINARY)
    mask = cv2.erode(th0,np.ones((7,7),np.uint8))

    output_path = os.path.join(outputFile,"Mask_{}".format(i))
    cv2.imwrite(output_path,mask)

print("Done")

Done
